In [2]:
# We want to create a token mask that we can easily access that is as smalla s possible, will be Binary 0 and 1s, so can multiply by the input, and the 0s are masked
#first load in the genome
import numpy as np

genome_sizes = {'chr1': 248956422,
'chr2':	242193529,
'chr3':	198295559,
'chr4':	190214555,
'chr5':	181538259,
'chr6':	170805979,
'chr7':	159345973,
'chr8':	145138636,
'chr9':	138394717,
'chr10':	133797422,
'chr11':	135086622,
'chr12':	133275309,
'chr13':	114364328,
'chr14':	107043718,
'chr15':	101991189,
'chr16':	90338345,
'chr17':	83257441,
'chr18':	80373285,
'chr19':	58617616,
'chr20':	64444167,
'chr21':	46709983,
'chr22':	50818468,
'chrX':	156040895,
'chrY':	57227415}
genome_sizes

{'chr1': 248956422,
 'chr2': 242193529,
 'chr3': 198295559,
 'chr4': 190214555,
 'chr5': 181538259,
 'chr6': 170805979,
 'chr7': 159345973,
 'chr8': 145138636,
 'chr9': 138394717,
 'chr10': 133797422,
 'chr11': 135086622,
 'chr12': 133275309,
 'chr13': 114364328,
 'chr14': 107043718,
 'chr15': 101991189,
 'chr16': 90338345,
 'chr17': 83257441,
 'chr18': 80373285,
 'chr19': 58617616,
 'chr20': 64444167,
 'chr21': 46709983,
 'chr22': 50818468,
 'chrX': 156040895,
 'chrY': 57227415}

In [3]:
#now create a dictionary of numpy arrays
genome = {}
for chrom in genome_sizes.keys():
    genome[chrom] = np.zeros(genome_sizes[chrom], dtype ='bool')
    print(chrom, genome[chrom].shape)

chr1 (248956422,)
chr2 (242193529,)
chr3 (198295559,)
chr4 (190214555,)
chr5 (181538259,)
chr6 (170805979,)
chr7 (159345973,)
chr8 (145138636,)
chr9 (138394717,)
chr10 (133797422,)
chr11 (135086622,)
chr12 (133275309,)
chr13 (114364328,)
chr14 (107043718,)
chr15 (101991189,)
chr16 (90338345,)
chr17 (83257441,)
chr18 (80373285,)
chr19 (58617616,)
chr20 (64444167,)
chr21 (46709983,)
chr22 (50818468,)
chrX (156040895,)
chrY (57227415,)


In [4]:
genome['chr1'][:10]

array([False, False, False, False, False, False, False, False, False,
       False])

In [7]:
nbytes = 0
for chrom in genome.keys():
    nbytes += genome[chrom].nbytes
nbytes/1e9 #about 3 GB as expected

3.088269832

In [8]:
#ok now we go through the bed file and we will turn it to ones/True if it's in the bed
import pandas as pd
file_path = '/data/leslie/sarthak/data/GRCh38-cCREs.bed'
df = pd.read_csv(file_path, sep = '\t', header = None)
df.head()

,0,1,2,3,4,5
0,chr1,104896,105048,EH38D4327509,EH38E2776520,"CTCF-only,CTCF-bound"
1,chr1,138866,139134,EH38D4327520,EH38E2776521,"pELS,CTCF-bound"
2,chr1,181289,181639,EH38D4327525,EH38E2776524,"DNase-H3K4me3,CTCF-bound"
3,chr1,267925,268171,EH38D4327544,EH38E2776528,"CTCF-only,CTCF-bound"
4,chr1,586036,586264,EH38D4327554,EH38E2776532,"CTCF-only,CTCF-bound"


In [9]:
#now go through each row
from tqdm import tqdm
for i in tqdm(range(df.shape[0]), total = df.shape[0]):
    chrom = df.iloc[i, 0]
    start = df.iloc[i, 1]
    end = df.iloc[i, 2]
    genome[chrom][start:end] = True

100%|██████████| 1063878/1063878 [00:48<00:00, 21840.03it/s]


In [10]:
#this is relatively quite fast, and now we can save this mask
np.savez('/data/leslie/sarthak/data/GRCh38-cCREs_mask.npz', **genome)

# and now altering mlm masking by adding this ccre mask

In [11]:
#this way can easily work with kmers or others
#so we load it in and get a sequence of 196608, let's test it
import torch
sample_seq = torch.ones((196608), dtype = torch.float16)
print(sample_seq.shape)

torch.Size([196608])


In [12]:
mlm_probability = 0.15
data = sample_seq.clone()
target = data.clone()
probability_matrix = torch.full(target.shape, mlm_probability)
print(probability_matrix.shape)

torch.Size([196608])


In [13]:
probability_matrix #ahh so just makes it the same size but the values are 0.15 each
#this is where we can apply our mask!

tensor([0.1500, 0.1500, 0.1500,  ..., 0.1500, 0.1500, 0.1500])

In [14]:
mask = torch.bernoulli(probability_matrix).bool()
print(mask)

tensor([ True, False, False,  ..., False,  True, False])


In [17]:
a = mask * probability_matrix
a

tensor([0.1500, 0.0000, 0.0000,  ..., 0.0000, 0.1500, 0.0000])

In [18]:
a.dtype #keeps the float type

torch.float32

In [19]:
bernoulli = torch.bernoulli(a).bool()
bernoulli

tensor([False, False, False,  ..., False, False, False])

In [21]:
print(sum(bernoulli)) #4376 out of 196608 is too low
print(sum(bernoulli)/bernoulli.shape[0]) #clearly below the 0.15
print(sum(bernoulli)/sum(a.bool())) #around the 15% that we expect, so this masking approach works!

tensor(4376)
tensor(0.0223)
tensor(0.1488)


# ROPE testing too

In [22]:
#we have the mask for cCREs now let's test ROPE
import torch.nn as nn
class RotaryPositionalEmbeddings(nn.Module):

  def __init__(self, d: int, base: int = 10_000):

    super().__init__()
    self.base = base
    self.d = d
    self.cos_cached = None
    self.sin_cached = None

  def _build_cache(self, x: torch.Tensor):

    if self.cos_cached is not None and x.shape[0] <= self.cos_cached.shape[0]:
      return

    seq_len = x.shape[0]

    theta = 1. / (self.base ** (torch.arange(0, self.d, 2).float() / self.d)).to(x.device) # THETA = 10,000^(-2*i/d) or 1/10,000^(2i/d)

    seq_idx = torch.arange(seq_len, device=x.device).float().to(x.device) #Position Index -> [0,1,2...seq-1]

    idx_theta = torch.einsum('n,d->nd', seq_idx, theta)  #Calculates m*(THETA) = [ [0, 0...], [THETA_1, THETA_2...THETA_d/2], ... [seq-1*(THETA_1), seq-1*(THETA_2)...] ]

    idx_theta2 = torch.cat([idx_theta, idx_theta], dim=1) # [THETA_1, THETA_2...THETA_d/2] -> [THETA_1, THETA_2...THETA_d]


    self.cos_cached = idx_theta2.cos()[:, None, None, :] #Cache [cosTHETA_1, cosTHETA_2...cosTHETA_d]
    self.sin_cached = idx_theta2.sin()[:, None, None, :] #cache [sinTHETA_1, sinTHETA_2...sinTHETA_d]

  def _neg_half(self, x: torch.Tensor):

    d_2 = self.d // 2 #

    return torch.cat([-x[:, :, :, d_2:], x[:, :, :, :d_2]], dim=-1) # [x_1, x_2,...x_d] -> [-x_d/2, ... -x_d, x_1, ... x_d/2]


  def forward(self, x: torch.Tensor):

    self._build_cache(x)

    neg_half_x = self._neg_half(x)

    x_rope = (x * self.cos_cached[:x.shape[0]]) + (neg_half_x * self.sin_cached[:x.shape[0]]) # [x_1*cosTHETA_1 - x_d/2*sinTHETA_d/2, ....]

    return x_rope

In [25]:
#let's see what happens when we do this
x = torch.rand((196608, 512), dtype = torch.float16)
rope = RotaryPositionalEmbeddings(512)
out = rope(x[:,None,None,:])[:,0,0,:]
out.shape

torch.Size([196608, 512])

In [26]:
print(x)

tensor([[0.2612, 0.0322, 0.9404,  ..., 0.0190, 0.5605, 0.0083],
        [0.4565, 0.1807, 0.9087,  ..., 0.5752, 0.4297, 0.9282],
        [0.5483, 0.5825, 0.9116,  ..., 0.3164, 0.5000, 0.1167],
        ...,
        [0.4546, 0.3862, 0.5557,  ..., 0.2559, 0.4946, 0.3105],
        [0.6841, 0.8496, 0.5161,  ..., 0.0098, 0.3491, 0.5835],
        [0.1152, 0.3120, 0.1182,  ..., 0.7456, 0.2114, 0.0273]],
       dtype=torch.float16)


In [27]:
print(out) #first row the same, rotated by 0, see it rotates as we go

tensor([[ 0.2612,  0.0322,  0.9404,  ...,  0.0190,  0.5605,  0.0083],
        [ 0.0684, -0.1788,  0.1877,  ...,  0.5753,  0.4298,  0.9282],
        [-0.8298, -0.4477, -0.9684,  ...,  0.3166,  0.5000,  0.1168],
        ...,
        [ 0.3846,  0.7037,  0.1268,  ..., -0.1932,  0.4112,  0.4675],
        [ 0.8043,  0.7064, -0.9518,  ...,  0.0035, -0.2216,  0.2719],
        [ 0.2510, -0.1667, -0.0868,  ..., -0.6952,  0.3114,  0.5583]])


In [32]:
def apply_rope(x: torch.Tensor, seq_len: int, dim: int) -> torch.Tensor:
    """
    Apply Rotary Position Embedding to input tensor.
    
    Args:
    x (torch.Tensor): Input tensor of shape (batch_size, seq_len, dim)
    seq_len (int): Sequence length
    dim (int): Dimension of the model (should be divisible by 2)
    
    Returns:
    torch.Tensor: Tensor with RoPE applied
    """
    device = x.device
    
    # Precompute RoPE matrix
    half_dim = dim // 2
    inv_freq = 1.0 / (10000 ** (torch.arange(0, half_dim, 2).float().to(device) / half_dim))
    t = torch.arange(seq_len, device=device).type_as(inv_freq)
    freqs = torch.einsum('i,j->ij', t, inv_freq)
    emb = torch.cat((freqs, freqs), dim=-1)
    cos = emb.cos().view(seq_len, 1, dim)
    sin = emb.sin().view(seq_len, 1, dim)
    
    # Rotate half of the dimensions
    x_rope = x.clone()
    x_pass, x_rot = x_rope.chunk(2, dim=-1)
    x_rot = torch.stack([-x_rot[..., 1::2], x_rot[..., ::2]], dim=-1).flatten(-2)
    x_rope = torch.cat((x_pass, x_rot), dim=-1)
    
    # Apply rotation
    return x_rope * cos + torch.roll(x_rope, shifts=1, dims=-1) * sin

In [39]:
#see the output based on RoPEMamba
apply_rope(x[None,:,:], seq_len=196608, dim=512)

RuntimeError: shape '[196608, 1, 512]' is invalid for input of size 50331648

In [40]:
#and now lucid rains implementation
from __future__ import annotations
from math import pi, log

import torch
from torch.nn import Module, ModuleList
from torch.amp import autocast
from torch import nn, einsum, broadcast_tensors, Tensor

from einops import rearrange, repeat

from typing import Literal

# helper functions

def exists(val):
    return val is not None

def default(val, d):
    return val if exists(val) else d

# broadcat, as tortoise-tts was using it

def broadcat(tensors, dim = -1):
    broadcasted_tensors = broadcast_tensors(*tensors)
    return torch.cat(broadcasted_tensors, dim = dim)

# rotary embedding helper functions

def rotate_half(x):
    x = rearrange(x, '... (d r) -> ... d r', r = 2)
    x1, x2 = x.unbind(dim = -1)
    x = torch.stack((-x2, x1), dim = -1)
    return rearrange(x, '... d r -> ... (d r)')

@autocast('cuda', enabled = False)
def apply_rotary_emb(freqs, t, start_index = 0, scale = 1., seq_dim = -2):
    dtype = t.dtype

    if t.ndim == 3:
        seq_len = t.shape[seq_dim]
        freqs = freqs[-seq_len:]

    rot_dim = freqs.shape[-1]
    end_index = start_index + rot_dim

    assert rot_dim <= t.shape[-1], f'feature dimension {t.shape[-1]} is not of sufficient size to rotate in all the positions {rot_dim}'

    # Split t into three parts: left, middle (to be transformed), and right
    t_left = t[..., :start_index]
    t_middle = t[..., start_index:end_index]
    t_right = t[..., end_index:]

    # Apply rotary embeddings without modifying t in place    
    t_transformed = (t_middle * freqs.cos() * scale) + (rotate_half(t_middle) * freqs.sin() * scale)
        
    out = torch.cat((t_left, t_transformed, t_right), dim=-1)

    return out.type(dtype)

# learned rotation helpers

def apply_learned_rotations(rotations, t, start_index = 0, freq_ranges = None):
    if exists(freq_ranges):
        rotations = einsum('..., f -> ... f', rotations, freq_ranges)
        rotations = rearrange(rotations, '... r f -> ... (r f)')

    rotations = repeat(rotations, '... n -> ... (n r)', r = 2)
    return apply_rotary_emb(rotations, t, start_index = start_index)

# classes

class RotaryEmbedding(Module):
    def __init__(
        self,
        dim,
        custom_freqs: Tensor | None = None,
        freqs_for:  Literal['lang', 'pixel', 'constant'] = 'lang',
        theta = 10000,
        max_freq = 10,
        num_freqs = 1,
        learned_freq = False,
        use_xpos = False,
        xpos_scale_base = 512,
        interpolate_factor = 1.,
        theta_rescale_factor = 1.,
        seq_before_head_dim = False,
        cache_if_possible = True,
        cache_max_seq_len = 8192
    ):
        super().__init__()
        # proposed by reddit user bloc97, to rescale rotary embeddings to longer sequence length without fine-tuning
        # has some connection to NTK literature
        # https://www.reddit.com/r/LocalLLaMA/comments/14lz7j5/ntkaware_scaled_rope_allows_llama_models_to_have/

        theta *= theta_rescale_factor ** (dim / (dim - 2))

        self.freqs_for = freqs_for

        if exists(custom_freqs):
            freqs = custom_freqs
        elif freqs_for == 'lang':
            freqs = 1. / (theta ** (torch.arange(0, dim, 2)[:(dim // 2)].float() / dim))
        elif freqs_for == 'pixel':
            freqs = torch.linspace(1., max_freq / 2, dim // 2) * pi
        elif freqs_for == 'constant':
            freqs = torch.ones(num_freqs).float()

        self.cache_if_possible = cache_if_possible
        self.cache_max_seq_len = cache_max_seq_len

        self.register_buffer('cached_freqs', torch.zeros(cache_max_seq_len, dim), persistent = False)
        self.register_buffer('cached_freqs_seq_len', torch.tensor(0), persistent = False)

        self.freqs = nn.Parameter(freqs, requires_grad = learned_freq)

        self.learned_freq = learned_freq

        # dummy for device

        self.register_buffer('dummy', torch.tensor(0), persistent = False)

        # default sequence dimension

        self.seq_before_head_dim = seq_before_head_dim
        self.default_seq_dim = -3 if seq_before_head_dim else -2

        # interpolation factors

        assert interpolate_factor >= 1.
        self.interpolate_factor = interpolate_factor

        # xpos

        self.use_xpos = use_xpos

        if not use_xpos:
            return

        scale = (torch.arange(0, dim, 2) + 0.4 * dim) / (1.4 * dim)
        self.scale_base = xpos_scale_base

        self.register_buffer('scale', scale, persistent = False)
        self.register_buffer('cached_scales', torch.zeros(cache_max_seq_len, dim), persistent = False)
        self.register_buffer('cached_scales_seq_len', torch.tensor(0), persistent = False)

        # add apply_rotary_emb as static method

        self.apply_rotary_emb = staticmethod(apply_rotary_emb)

    @property
    def device(self):
        return self.dummy.device

    def get_seq_pos(self, seq_len, device, dtype, offset = 0):
        return (torch.arange(seq_len, device = device, dtype = dtype) + offset) / self.interpolate_factor

    def rotate_queries_or_keys(self, t, seq_dim = None, offset = 0, scale = None):
        seq_dim = default(seq_dim, self.default_seq_dim)

        assert not self.use_xpos or exists(scale), 'you must use `.rotate_queries_and_keys` method instead and pass in both queries and keys, for length extrapolatable rotary embeddings'

        device, dtype, seq_len = t.device, t.dtype, t.shape[seq_dim]

        seq = self.get_seq_pos(seq_len, device = device, dtype = dtype, offset = offset)

        freqs = self.forward(seq, seq_len = seq_len, offset = offset)

        if seq_dim == -3:
            freqs = rearrange(freqs, 'n d -> n 1 d')

        return apply_rotary_emb(freqs, t, scale = default(scale, 1.), seq_dim = seq_dim)

    def rotate_queries_with_cached_keys(self, q, k, seq_dim = None, offset = 0):
        dtype, device, seq_dim = q.dtype, q.device, default(seq_dim, self.default_seq_dim)

        q_len, k_len = q.shape[seq_dim], k.shape[seq_dim]
        assert q_len <= k_len

        q_scale = k_scale = 1.

        if self.use_xpos:
            seq = self.get_seq_pos(k_len, dtype = dtype, device = device)

            q_scale = self.get_scale(seq[-q_len:]).type(dtype)
            k_scale = self.get_scale(seq).type(dtype)

        rotated_q = self.rotate_queries_or_keys(q, seq_dim = seq_dim, scale = q_scale, offset = k_len - q_len + offset)
        rotated_k = self.rotate_queries_or_keys(k, seq_dim = seq_dim, scale = k_scale ** -1)

        rotated_q = rotated_q.type(q.dtype)
        rotated_k = rotated_k.type(k.dtype)

        return rotated_q, rotated_k

    def rotate_queries_and_keys(self, q, k, seq_dim = None):
        seq_dim = default(seq_dim, self.default_seq_dim)

        assert self.use_xpos
        device, dtype, seq_len = q.device, q.dtype, q.shape[seq_dim]

        seq = self.get_seq_pos(seq_len, dtype = dtype, device = device)

        freqs = self.forward(seq, seq_len = seq_len)
        scale = self.get_scale(seq, seq_len = seq_len).to(dtype)

        if seq_dim == -3:
            freqs = rearrange(freqs, 'n d -> n 1 d')
            scale = rearrange(scale, 'n d -> n 1 d')

        rotated_q = apply_rotary_emb(freqs, q, scale = scale, seq_dim = seq_dim)
        rotated_k = apply_rotary_emb(freqs, k, scale = scale ** -1, seq_dim = seq_dim)

        rotated_q = rotated_q.type(q.dtype)
        rotated_k = rotated_k.type(k.dtype)

        return rotated_q, rotated_k

    def get_scale(
        self,
        t: Tensor,
        seq_len: int | None = None,
        offset = 0
    ):
        assert self.use_xpos

        should_cache = (
            self.cache_if_possible and
            exists(seq_len) and
            (offset + seq_len) <= self.cache_max_seq_len
        )

        if (
            should_cache and \
            exists(self.cached_scales) and \
            (seq_len + offset) <= self.cached_scales_seq_len.item()
        ):
            return self.cached_scales[offset:(offset + seq_len)]

        scale = 1.
        if self.use_xpos:
            power = (t - len(t) // 2) / self.scale_base
            scale = self.scale ** rearrange(power, 'n -> n 1')
            scale = repeat(scale, 'n d -> n (d r)', r = 2)

        if should_cache and offset == 0:
            self.cached_scales[:seq_len] = scale.detach()
            self.cached_scales_seq_len.copy_(seq_len)

        return scale

    def get_axial_freqs(self, *dims):
        Colon = slice(None)
        all_freqs = []

        for ind, dim in enumerate(dims):
            if self.freqs_for == 'pixel':
                pos = torch.linspace(-1, 1, steps = dim, device = self.device)
            else:
                pos = torch.arange(dim, device = self.device)

            freqs = self.forward(pos, seq_len = dim)

            all_axis = [None] * len(dims)
            all_axis[ind] = Colon

            new_axis_slice = (Ellipsis, *all_axis, Colon)
            all_freqs.append(freqs[new_axis_slice])

        all_freqs = broadcast_tensors(*all_freqs)
        return torch.cat(all_freqs, dim = -1)

    @autocast('cuda', enabled = False)
    def forward(
        self,
        t: Tensor,
        seq_len = None,
        offset = 0
    ):
        should_cache = (
            self.cache_if_possible and
            not self.learned_freq and
            exists(seq_len) and
            self.freqs_for != 'pixel' and
            (offset + seq_len) <= self.cache_max_seq_len
        )

        if (
            should_cache and \
            exists(self.cached_freqs) and \
            (offset + seq_len) <= self.cached_freqs_seq_len.item()
        ):
            return self.cached_freqs[offset:(offset + seq_len)].detach()

        freqs = self.freqs

        freqs = einsum('..., f -> ... f', t.type(freqs.dtype), freqs)
        freqs = repeat(freqs, '... n -> ... (n r)', r = 2)

        if should_cache and offset == 0:
            self.cached_freqs[:seq_len] = freqs.detach()
            self.cached_freqs_seq_len.copy_(seq_len)

        return freqs

In [ ]:
rotary_emb = RotaryEmbedding(dim = 512)


In [43]:
#I can do this, but this really seems to be for attention, why don't we just do the good old absolute embeddings, we already have relative
def positionalencoding1d(d_model, length):
    """
    :param d_model: dimension of the model
    :param length: length of positions
    :return: length*d_model position matrix
    """
    if d_model % 2 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dim (got dim={:d})".format(d_model))
    pe = torch.zeros(length, d_model)
    position = torch.arange(0, length).unsqueeze(1)
    div_term = torch.exp((torch.arange(0, d_model, 2, dtype=torch.float) *
                         -(torch.log(torch.tensor(10000)) / d_model)))
    pe[:, 0::2] = torch.sin(position.float() * div_term)
    pe[:, 1::2] = torch.cos(position.float() * div_term)

    return pe

pos_enc = positionalencoding1d(512, 196608)

In [44]:
pos_enc.shape

torch.Size([196608, 512])

In [45]:
#ok so it's the right size
pos_enc

tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  8.2186e-01,  ...,  1.0000e+00,
          1.0366e-04,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  9.3641e-01,  ...,  1.0000e+00,
          2.0733e-04,  1.0000e+00],
        ...,
        [-8.3611e-01, -5.4857e-01, -6.0648e-01,  ..., -6.4953e-01,
          9.9922e-01,  3.9614e-02],
        [-9.1335e-01,  4.0717e-01,  3.1184e-01,  ..., -6.4961e-01,
          9.9922e-01,  3.9509e-02],
        [-1.5087e-01,  9.8855e-01,  9.5518e-01,  ..., -6.4970e-01,
          9.9922e-01,  3.9406e-02]])

In [46]:
#I think this is better and makes more sense
#So let's see what the part after the embedding looks like
ckpt_path = '/data/leslie/sarthak/caduceus/outputs/2024-09-18/13-52-33-428792/checkpoints/last.ckpt'
# we have lots of model outputs, now let's test it to see what works
import torch
import sys
sys.path.append('/data/leslie/sarthak/caduceus')
import evals.evals_utils_enformer as e
#now let's load the model anad we can plot some things
import numpy as np
split = 'test'

labels = np.load(f'/data/leslie/sarthak/data/enformer/data/{split}_label.npy')

evals = e.Evals(ckpt_path)

Using kmer genome with length 6
4675


In [47]:
evals.backbone

DNAEmbeddingModelCaduceus(
  (caduceus): Caduceus(
    (backbone): CaduceusMixerModel(
      (embeddings): CaduceusEmbeddings(
        (word_embeddings): RCPSEmbedding(
          (embedding): Embedding(15632, 256)
        )
      )
      (layers): ModuleList(
        (0-9): 10 x RCPSMambaBlock(
          (mixer): RCPSWrapper(
            (submodule): BiMambaWrapper(
              (mamba_fwd): Mamba(
                (in_proj): Linear(in_features=256, out_features=1024, bias=False)
                (conv1d): Conv1d(512, 512, kernel_size=(4,), stride=(1,), padding=(3,), groups=512)
                (act): SiLU()
                (x_proj): Linear(in_features=512, out_features=48, bias=False)
                (dt_proj): Linear(in_features=16, out_features=512, bias=True)
                (out_proj): Linear(in_features=512, out_features=256, bias=False)
              )
              (mamba_rev): Mamba(
                (in_proj): Linear(in_features=256, out_features=1024, bias=False)
             

In [49]:
evals.backbone.caduceus.backbone.embeddings

CaduceusEmbeddings(
  (word_embeddings): RCPSEmbedding(
    (embedding): Embedding(15632, 256)
  )
)

In [51]:
evals.backbone.caduceus.backbone.embeddings.word_embeddings.embedding

Embedding(15632, 256)

In [54]:
#so we see that the embedding is just d_model...
#so let's use random integers of length 196608
input = torch.randint(0, 21, (1,196608))
input.shape
# evals.backbone.caduceus.backbone.embeddings()

torch.Size([1, 196608])

In [56]:
evals.backbone.caduceus.backbone.embeddings(input.to('cuda')).shape

torch.Size([1, 196608, 512])

In [ ]:
#so we can see that the embedding already doubles it, so we have to set it up so that the RCPS Embedding which we use takes into account the positional embedding